In [202]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta, date
import random, unicodedata
from decimal import Decimal
from itertools import cycle

łączenie do serwera

In [204]:
username = "team01"
password = quote_plus("te@mloi")  # koduje hasło bezpiecznie
host = "giniewicz.it"
port = 3306
database = "team01"

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
)

fake = Faker("pl_PL")
today = datetime.today()

USUWANIE WSZYSTKICH DANYCH JEŚLI COŚ TRZEBA POPRAWIĆ

In [205]:
 # Lista tabel w odpowiedniej kolejności usuwania (najpierw zależne)
 tables = [
     "Opinie_Klientow",
     "Rezerwacje",
     "Wyprawy",
     "Kontakty_Awaryjne",
     "Klienci",
     "Rodzaje_Wypraw",
     "Kierunki_Wypraw",
     "Pojazdy_Kosmiczne",
     "Typy_Pojazdow",
     "Rakiety",
     "Obsluga",
     "Wspieramy",
     "Partnerzy_Strategiczni",
     "Sponsorzy"
 ]

 with engine.begin() as conn:
     print("🧨 Wyłączam klucze obce...")
     conn.execute(text("SET FOREIGN_KEY_CHECKS = 0"))

     for table in tables:
         print(f"🧹 Czyszczenie tabeli: {table}")
         conn.execute(text(f"DELETE FROM {table}"))

     print("🔒 Włączam z powrotem klucze obce...")
     conn.execute(text("SET FOREIGN_KEY_CHECKS = 1"))

 print("✅ Wszystkie dane zostały usunięte.")

🧨 Wyłączam klucze obce...
🧹 Czyszczenie tabeli: Opinie_Klientow
🧹 Czyszczenie tabeli: Rezerwacje
🧹 Czyszczenie tabeli: Wyprawy
🧹 Czyszczenie tabeli: Kontakty_Awaryjne
🧹 Czyszczenie tabeli: Klienci
🧹 Czyszczenie tabeli: Rodzaje_Wypraw
🧹 Czyszczenie tabeli: Kierunki_Wypraw
🧹 Czyszczenie tabeli: Pojazdy_Kosmiczne
🧹 Czyszczenie tabeli: Typy_Pojazdow
🧹 Czyszczenie tabeli: Rakiety
🧹 Czyszczenie tabeli: Obsluga
🧹 Czyszczenie tabeli: Wspieramy
🧹 Czyszczenie tabeli: Partnerzy_Strategiczni
🧹 Czyszczenie tabeli: Sponsorzy
🔒 Włączam z powrotem klucze obce...
✅ Wszystkie dane zostały usunięte.


### Obsługa

In [206]:
N_PRACOWNICY = 14

stanowiska = (
    ["Pilot"] * 3 +
    ["Operator"] * 3 +
    ["Mechanik"] * 2 +
    ["Analityk"] * 2 +
    ["Szkoleniowiec"] * 2 +
    ["Technik"] * 2
)

def strip_pl(txt: str) -> str:
    return unicodedata.normalize("NFKD", txt).encode("ascii", "ignore").decode("ascii")

def pesel():
    b = fake.date_of_birth(minimum_age=25, maximum_age=60)  
    y, m, d = b.year, b.month, b.day
    if 2000 <= y < 2100: m += 20
    elif 2100 <= y < 2200: m += 40
    elif 1800 <= y < 1900: m += 80
    base = f"{str(y)[2:]}{m:02d}{d:02d}{random.randint(0,9999):04d}"
    w = [1,3,7,9,1,3,7,9,1,3]
    ctrl = (10 - sum(int(base[i])*w[i] for i in range(10)) % 10) % 10
    return base + str(ctrl)

def email_for(first, last):
    return f"{strip_pl(first).lower()}.{strip_pl(last).lower()}@" \
           f"{random.choice(['gmail.com','onet.pl','wp.pl','interia.pl'])}"

with engine.begin() as conn:
    for idx in range(N_PRACOWNICY):
        imie_pl  = fake.first_name()
        nazw_pl  = fake.last_name()
        conn.execute(text("""
            INSERT INTO Obsluga
                (id_pracownika, imie, nazwisko, pesel,
                 stanowisko, wynagrodzenie, email, aktywny)
            VALUES
                (:id, :imie, :nazwisko, :pesel,
                 :stanowisko, :wyn, :email, :aktywny)
        """), {
            "id":         idx + 1,
            "imie":       imie_pl,            
            "nazwisko":   nazw_pl,              
            "pesel":      pesel(),
            "stanowisko": stanowiska[idx],
            "wyn":        random.randint(4800, 12000),
            "email":      email_for(imie_pl, nazw_pl),  
            "aktywny":    True
        })

print("pracownicy dodani")

pracownicy dodani


### Klienci i Kontakty_Awaryjne

In [207]:
N_KLIENCI = 35

def strip_pl(txt: str) -> str:
    """usuń diakrytyki (ą -> a, ł -> l, …)"""
    return unicodedata.normalize("NFKD", txt).encode("ascii", "ignore").decode("ascii")

def generate_pesel():
    b = fake.date_of_birth(minimum_age=18, maximum_age=70)
    y, m, d = b.year, b.month, b.day
    if 2000 <= y < 2100: m += 20
    elif 2100 <= y < 2200: m += 40
    elif 1800 <= y < 1900: m += 80
    base = f"{str(y)[2:]}{m:02d}{d:02d}{random.randint(0,9999):04d}"
    w = [1,3,7,9,1,3,7,9,1,3]
    ctrl = (10 - sum(int(base[i])*w[i] for i in range(10)) % 10) % 10
    return base + str(ctrl)

def generate_email(imie, nazwisko):
    domeny = ['gmail.com', 'onet.pl', 'wp.pl', 'interia.pl']
    return f"{strip_pl(imie).lower()}.{strip_pl(nazwisko).lower()}@{random.choice(domeny)}"

def generate_tel():
    return "48" + fake.msisdn()[-9:]

with engine.begin() as conn:
    for i in range(1, N_KLIENCI + 1):
        plec = random.choice(["M", "K"])
        imie = fake.first_name_male() if plec == "M" else fake.first_name_female()
        nazwisko = fake.last_name()

        conn.execute(text("""
            INSERT INTO Klienci
                (id_klienta, imie, nazwisko, pesel, email, telefon, wyniki_badan)
            VALUES
                (:id, :imie, :nazwisko, :pesel, :email, :tel, :zdrowie)
        """), {
            "id":       i,
            "imie":     imie,
            "nazwisko": nazwisko,
            "pesel":    generate_pesel(),
            "email":    generate_email(imie, nazwisko),  
            "tel":      generate_tel(),
            "zdrowie":  random.choice(['dobry', 'zły', 'fatalny'])
        })

        relacje_all = ["ojciec","matka","mąż","żona","brat","siostra","kolega","przyjaciel"]
        relacje = [r for r in relacje_all if not (plec=="M" and r=="mąż") and not (plec=="K" and r=="żona")]
        relacja = random.choice(relacje)

        if relacja in {"matka","siostra","żona"}:  k_imie = fake.first_name_female()
        elif relacja in {"ojciec","brat","mąż"}:   k_imie = fake.first_name_male()
        else:                                      k_imie = fake.first_name()

        k_nazwisko = nazwisko if relacja in {"ojciec","matka","mąż","żona","brat","siostra"} else fake.last_name()

        conn.execute(text("""
            INSERT INTO Kontakty_Awaryjne
                (id_kontaktu, id_klienta, imie, nazwisko, stopien_pokrewienstwa, telefon)
            VALUES
                (:id_k, :id_c, :imie, :nazwisko, :relacja, :tel)
        """), {
            "id_k":    i,
            "id_c":    i,
            "imie":    k_imie,
            "nazwisko":k_nazwisko,
            "relacja": relacja,
            "tel":     generate_tel()
        })

print("klienci i kontakty awaryjne dodane")

klienci i kontakty awaryjne dodane


### Typy_Pojazdów

In [208]:
typy_pojazdow = [
    (1, "Orion Tourist Cruiser",
     "Średniej wielkości wahadłowiec na loty turystyczne wokół Księżyca; panoramiczne kopuły obserwacyjne.",
     12, 500_000, 28_000),

    (2, "Starlight Hopper",
     "Lekki pojazd suborbitalny do krótkich lotów typu „zero-G experience” z szybkim powrotem na Ziemię.",
     18, 1_000, 3_500),

    (3, "Pegasus Luxury Liner",
     "Luksusowy liner umożliwiający kilkudniowe pobyty w niskiej orbicie z pełnym zapleczem rekreacyjnym.",
     30, 800_000, 30_000),

    (4, "Red Dragon Explorer",
     "Statek dalekiego zasięgu zoptymalizowany pod przeloty i orbitę Marsa; systemy życia na 60 dni.",
     8, 400_000_000, 45_000),

    (5, "Europa Glider",
     "Kompaktowy lądownik do lotów zwiadowczych w układzie Jowisza – głównie wokół księżyca Europa.",
     4, 600_000_000, 52_000),

    (6, "Asteroid Miner AX-5",
     "Wzmocniony kadłub i moduły górnicze – turystyczno-badawcze wyprawy do pasa asteroid i symulacje wydobycia.",
     6, 1_000_000_000, 50_000),
]

with engine.begin() as conn:
    for (id_typu, nazwa, opis, pojemnosc, zasieg, predkosc) in typy_pojazdow:
        conn.execute(text("""
            INSERT INTO Typy_Pojazdow
                (id_typu, nazwa, opis, pojemnosc_pasazerow, zasieg_km, predkosc_max_kmh)
            VALUES
                (:id_typu, :nazwa, :opis, :pojemnosc, :zasieg, :predkosc)
        """), {
            "id_typu":    id_typu,
            "nazwa":      nazwa,
            "opis":       opis,
            "pojemnosc":  pojemnosc,
            "zasieg":     zasieg,
            "predkosc":   predkosc,
        })

print("typy pojazdów dodane")


typy pojazdów dodane


### Rakiety

In [209]:
rakiety = [
    # id,     nazwa,                 producent,         udźwig_kg, koszt_startu (PLN), liczba_użyć, status
    (1, "Super Heavy Starship",      "Space-U",          150_000,   12_000_000,          0,          "aktywny"),
    (2, "Solaris Heavy",             "Helios Dynamics",   65_000,    6_500_000,          0,          "aktywny"),
    (3, "Ariane Next",               "EuroLaunch",        18_000,    4_800_000,          0,          "aktywny"),
    (4, "Nova-C Pathfinder",         "LunaWorks",          9_000,    3_200_000,          0,          "aktywny"),
    (5, "Atlas-X Classic",           "Orbital Systems",    7_000,    2_200_000,         12,          "wycofany"),
    (6, "Red Booster M-II",          "Mars Horizons",     28_000,    5_900_000,          0,          "aktywny"),
]

with engine.begin() as conn:
    for (id_rakiety, nazwa, producent,
         udzwig, koszt, uzyc, status) in rakiety:
        conn.execute(text("""
            INSERT INTO Rakiety
                (id_rakiety, nazwa, producent, udzwig_kg,
                 koszt_startu, liczba_uzyc, status_operacyjny)
            VALUES
                (:id_rakiety, :nazwa, :producent, :udzwig,
                 :koszt, :uzyc, :status)
        """), {
            "id_rakiety": id_rakiety,
            "nazwa":       nazwa,
            "producent":   producent,
            "udzwig":      udzwig,
            "koszt":       koszt,
            "uzyc":        uzyc,
            "status":      status,
        })

print("rakiety dodane")


rakiety dodane


### Kierunki_Wypraw

In [210]:
miejsca = [
    (1, "LEO – Międzynarodowa Stacja Kosmiczna", "orbita", 0.000, 0.000, 0.000, 408, 0.00, "niski"),
    (2, "Orbita Księżyca",                       "orbita", 384_400.000, 0.000, 0.000, 384_400, 0.00, "średni"),
    (3, "Mare Tranquillitatis – Księżyc",        "ksiezyc", 384_400.000, 50.000, 10.000, 384_400, 0.17, "średni"),
    (4, "Orbita Marsa",                          "orbita", 78_000_000.000, 0.000, 0.000, 78_000_000, 0.00, "wysoki"),
    (5, "Kolonia Phobos One",                    "ksiezyc", 78_000_000.000, 9_378.000, 800.000, 78_000_000, 0.01, "wysoki"),
    (6, "Pas asteroid – Sektor 14",              "pas asteroid", 300_000_000.000, 50_000_000.000, -1_000_000.000, 300_000_000, 0.00, "ekstremalny"),
    (7, "Europa Orbital Station",                "stacja", 628_300_000.000, 0.000, 5_000.000, 628_300_000, 0.13, "ekstremalny"),
]

with engine.begin() as conn:
    for (id_miejsca, nazwa, typ, x, y, z, odleglosc, g, ryzyko) in miejsca:
        conn.execute(text("""
            INSERT INTO Kierunki_Wypraw
                (id_miejsca, nazwa, typ,
                 wspolrzedne_x, wspolrzedne_y, wspolrzedne_z,
                 odleglosc_od_ziemi_km, grawitacja_g, poziom_ryzyka)
            VALUES
                (:id_miejsca, :nazwa, :typ,
                 :x, :y, :z, :odleglosc, :g, :ryzyko)
        """), {
            "id_miejsca": id_miejsca,
            "nazwa":       nazwa,
            "typ":         typ,
            "x":           x,
            "y":           y,
            "z":           z,
            "odleglosc":   odleglosc,
            "g":           g,
            "ryzyko":      ryzyko,
        })

print("Kierunki dodane")


Kierunki dodane


### Pojazdy_Kosmiczne

In [211]:
vehicles = [
    # id   nazwa              id_typu  data_prod     ostatni_przeglad  status
    (1,  "Selene One",         1,     "2024-03-15", "2025-03-20",     "aktywny"),
    (2,  "Luna Vista",         1,     "2024-04-02", "2025-02-28",     "aktywny"),
    (3,  "Zero-Two",           2,     "2023-07-19", "2025-01-12",     "aktywny"),
    (4,  "Celestia",           3,     "2025-01-05", "2025-06-01",     "aktywny"),
    (5,  "Ares Voyager",       4,     "2024-09-11", "2025-05-15",     "aktywny"),   
    (6,  "Europa Dream",       5,     "2025-02-20", "2025-05-25",     "aktywny"),
    (7,  "Miner One",          6,     "2024-06-30", "2025-04-10",     "w naprawie"),
    (8,  "Ares Pathfinder",    4,     "2024-10-22", "2025-06-01",     "aktywny"),   
    (9,  "Red Horizon",        4,     "2023-12-01", "2025-05-10",     "aktywny"),   
    (10, "Starlight Nova",     2,     "2023-05-18", "2025-03-30",     "aktywny"),
    (11, "Orion Explorer",     1,     "2024-05-01", "2025-04-25",     "aktywny"),
    (12, "Pegasus Serenity",   3,     "2025-02-10", "2025-06-05",     "aktywny"),
]

with engine.begin() as conn:
    for (id_pojazdu, nazwa, id_typu,
         data_prod, przeglad, status) in vehicles:
        conn.execute(text("""
            INSERT INTO Pojazdy_Kosmiczne
                (id_pojazdu, nazwa, id_typu,
                 data_produkcji, ostatni_przeglad, status_operacyjny)
            VALUES
                (:id, :nazwa, :typ,
                 :prod, :przeglad, :status)
        """), {
            "id":        id_pojazdu,
            "nazwa":     nazwa,
            "typ":       id_typu,
            "prod":      data_prod,
            "przeglad":  przeglad,
            "status":    status,
        })

print("pojazdy kosmiczne dodane")

pojazdy kosmiczne dodane


### Rodzaje_Wypraw

In [212]:
rodzaje = [
    # id  nazwa  opis  dni  id_miejsca  cena trudnosc
    (1, "Księżyc – Przelot Orbitalny",
        "25-godzinny lot i 10 okrążeń wokół Księżyca w panoramicznym cruiserze.",
        3,   2,          950_000.00, "umiarkowany"),

    (2, "Księżyc – Lądowanie Tranquillitatis",
        "Lądowanie w Mare Tranquillitatis, dwie noce w module księżycowym i spacer EVA.",
        5,   3,        1_200_000.00, "trudny"),

    (3, "Zero-G Experience",
        "Suborbitalny skok na wysokość 100 km – 5 minut nieważkości i szybki powrót.",
        1,   1,           12_000.00, "łatwy"),

    (4, "Hotel Orbitalny ISS+",
        "Czterodniowy pobyt w luksusowym module hotelowym na LEO, pełna panorama Ziemi.",
        4,   1,          250_000.00, "umiarkowany"),

    (5, "Mars Orbiter Adventure",
        "6-miesięczna misja do orbity Marsa z przelotem przez punkt Lagrange’a i oknem foto.",
        180, 4,        7_500_000.00, "trudny"),

    (6, "Phobos Explorer",
        "Bliski przelot i dwudniowe badania na księżycu Phobos – ograniczona grawitacja.",
        190, 5,        8_800_000.00, "ekstremalny"),

    (7, "Asteroid Mining Mission S14",
        "Turystyczno-badawcza wyprawa do pasa asteroid; symulacja wydobycia regolitu.",
        270, 6,       12_000_000.00, "ekstremalny"),

    (8, "Europa Science Cruise",
        "Roczna ekspedycja orbitalna wokół Europy z zdalnym sterowaniem sond podlodowych.",
        400, 7,       18_000_000.00, "ekstremalny"),
]

with engine.begin() as conn:
    for (id_rodzaju, nazwa, opis, dni, id_miejsca,
         cena, trudnosc) in rodzaje:
        conn.execute(text("""
            INSERT INTO Rodzaje_Wypraw
                (id_rodzaju, nazwa, opis, czas_trwania_dni,
                 id_miejsca_docelowego, cena_bazowa, poziom_trudnosci)
            VALUES
                (:id, :nazwa, :opis, :dni,
                 :miejsce, :cena, :trudnosc)
        """), {
            "id":        id_rodzaju,
            "nazwa":     nazwa,
            "opis":      opis,
            "dni":       dni,
            "miejsce":   id_miejsca,
            "cena":      cena,
            "trudnosc":  trudnosc,
        })

print("rodzaje wypraw dodane")

rodzaje wypraw dodane


### Wyprawy

In [213]:
with engine.begin() as conn:
    pilot_ids = [row[0] for row in conn.execute(text("""
        SELECT id_pracownika
        FROM   Obsluga
        WHERE  stanowisko = 'Pilot'
    """)).all()]

if not pilot_ids:
    raise ValueError("Brak pracowników ze stanowiskiem 'Pilot' – dodaj pilota do tabeli Pracownicy!")


pilot_cycle = cycle(pilot_ids)

wyprawy = [
    # id  rodzaj  pojazd  rakieta           start →          powrót →            status
    (1,   3,      10,     2,      "2025-06-15 09:00:00", "2025-06-15 15:00:00", "zakończona"),
    (2,   1,       1,     1,      "2025-05-01 08:00:00", "2025-05-04 18:00:00", "zakończona"),
    (3,   4,       4,     1,      "2025-04-10 07:30:00", "2025-04-14 20:00:00", "zakończona"),
    (4,   2,       2,     1,      "2025-03-05 09:00:00", "2025-03-10 22:00:00", "zakończona"),
    (5,   5,       5,     6,      "2024-11-01 12:00:00", "2025-05-01 14:00:00", "zakończona"),
    (6,   6,       8,     6,      "2024-10-15 10:00:00", "2025-04-23 11:00:00", "zakończona"),
    (7,   7,      12,     6,      "2024-08-01 13:00:00", "2025-05-28 16:00:00", "zakończona"),
    (8,   3,      11,     2,      "2025-07-01 09:00:00", "2025-07-01 15:00:00", "zaplanowana"),
    (9,   1,       6,     1,      "2025-07-20 08:30:00", "2025-07-23 19:00:00", "zaplanowana"),
    (10,  8,       9,     6,      "2025-01-10 11:00:00", "2026-02-14 13:00:00", "w trakcie"),
]

with engine.begin() as conn:
    for (id_wyprawy, id_rodzaju, id_pojazdu, id_rakiety,
         start, koniec, status) in wyprawy:

        id_pilota = next(pilot_cycle)   

        conn.execute(text("""
            INSERT INTO Wyprawy
                (id_wyprawy, id_rodzaju_wyprawy, id_pojazdu, id_rakiety, id_pilota,
                 data_startu, data_powrotu, status_wyprawy)
            VALUES
                (:id_wyprawy, :rodzaj, :pojazd, :rakieta, :pilot,
                 :start, :koniec, :status)
        """), {
            "id_wyprawy": id_wyprawy,
            "rodzaj":     id_rodzaju,
            "pojazd":     id_pojazdu,
            "rakieta":    id_rakiety,
            "pilot":      id_pilota,
            "start":      start,
            "koniec":     koniec,
            "status":     status,
        })

print(f"wyprawy dodane")

wyprawy dodane


### Koszty

In [214]:
koszty = [
    # id  wyprawa  kategoria   opis                                   kwota        data (DATE!)
    (1,  1, "paliwo",        "Metan + LOX – tankowanie",              320_000.00, date(2025, 6, 14)),
    (2,  1, "ubezpieczenie", "Polisa lotu suborbitalnego",             22_000.00, date(2025, 6, 10)),
    (3,  2, "paliwo",        "Kerozyna – start na LEO",               410_000.00, date(2025, 4, 28)),
    (4,  2, "logistyka",     "Obsługa wieży + tracking",               55_500.00, date(2025, 5,  2)),
    (5,  3, "paliwo",        "Metan + LOX – tankowanie",              330_000.00, date(2025, 4,  8)),
    (6,  3, "sprzet",        "Przegląd silników Raptor",               75_000.00, date(2025, 4, 18)),
    (7,  4, "paliwo",        "Kerozyna – start na Księżyc",           550_000.00, date(2025, 3,  2)),
    (8,  4, "inne",          "Symulator EVA (2 dni)",                  18_600.00, date(2025, 2, 27)),
    (9,  5, "paliwo",        "Metan – tranzyt Mars LEO→TMI",          920_000.00, date(2024,10, 29)),
    (10, 5, "ubezpieczenie", "Polisa misji międzyplanetarnej",        130_000.00, date(2024,10, 25)),

    (11, 6, "paliwo",        "Metan – manewr marsko-fobosowy",        810_000.00, date(2024,10, 12)),
    (12, 6, "logistyka",     "Deep-space network łącze",              290_000.00, date(2024,10, 14)),
    (13, 7, "paliwo",        "Kerozyna – start do pasa asteroid",   1_150_000.00, date(2024, 7, 28)),
    (14, 7, "sprzet",        "Wymiana dysz ablacyjnych",               95_000.00, date(2025, 1, 16)),
    (15, 8, "paliwo",        "Suborbitalny skok – metan",              20_000.00, date(2025, 6, 29)),
    (16, 8, "logistyka",     "Wieża + odzysk kapsuły",                 12_300.00, date(2025, 7,  1)),
    (17, 9, "paliwo",        "Kerozyna – orbit hop",                   60_000.00, date(2025, 7, 17)),
    (18, 9, "ubezpieczenie", "Polisa orbitalna",                       15_500.00, date(2025, 7, 15)),
    (19,10, "paliwo",        "Metan – transfer Jowisz→Europa",      1_800_000.00, date(2025, 1,  6)),
    (20,10, "sprzet",        "Upgrade systemu life-support",          250_000.00, date(2024,12, 20)),
]

with engine.begin() as conn:
    conn.execute(text("DELETE FROM Koszty"))

    for (id_kosztu, id_wyprawy, kat, opis, kwota, d) in koszty:
        conn.execute(text("""
            INSERT INTO Koszty
                (id_kosztu, id_wyprawy, kategoria, opis, kwota, data_poniesienia)
            VALUES
                (:id, :wyprawa, :kat, :opis, :kwota, :d)
        """), {
            "id":      id_kosztu,
            "wyprawa": id_wyprawy,
            "kat":     kat,
            "opis":    opis,
            "kwota":   kwota,
            "d":       d,
        })

print("koszty dodane")

koszty dodane


### Rezerwacje

In [215]:
with engine.begin() as conn:
    rows = conn.execute(text("""
        SELECT w.id_wyprawy, r.cena_bazowa
        FROM   Wyprawy w
        JOIN   Rodzaje_Wypraw r
               ON w.id_rodzaju_wyprawy = r.id_rodzaju
    """)).all()

ceny = {row[0]: row[1] for row in rows}  

def enum_values(table, column):
    with engine.begin() as conn:
        raw = conn.execute(text(f"""
            SELECT COLUMN_TYPE
            FROM   information_schema.COLUMNS
            WHERE  TABLE_SCHEMA = DATABASE()
              AND  TABLE_NAME   = '{table}'
              AND  COLUMN_NAME  = '{column}'
        """)).scalar()
    return [s.strip("'") for s in raw[5:-1].split(",")]

statusy = enum_values("Rezerwacje", "status_platnosci")
metody  = enum_values("Rezerwacje", "metoda_platnosci")

id_klientow  = list(range(1, 16))       
id_rez       = 1
rezers       = []

for id_wyprawy, cena_baz in ceny.items():
    for _ in range(random.randint(3, 5)):        
        klient   = random.choice(id_klientow)
        miejsca  = random.randint(1, 4)
        rabat    = Decimal(str(round(random.uniform(0, 0.10), 4)))   
        cena     = (cena_baz * Decimal(miejsca) * (Decimal(1) - rabat))\
                   .quantize(Decimal("0.01"))
        data_rez = datetime(2024, 1, 1) + timedelta(days=random.randint(0, 400))
        status   = random.choice(statusy)
        metoda   = random.choice(metody)

        rezers.append((id_rez, klient, id_wyprawy, data_rez,
                       cena, status, metoda))
        id_rez += 1

with engine.begin() as conn:
    for (idr, idk, idw, data_r, cena, stat, metoda) in rezers:
        conn.execute(text("""
            INSERT INTO Rezerwacje
                (id_rezerwacji, id_klienta, id_wyprawy, data_rezerwacji,
                 cena_koncowa, status_platnosci, metoda_platnosci)
            VALUES
                (:idr, :idk, :idw, :data_r, :cena, :stat, :metoda)
        """), {
            "idr":    idr,
            "idk":    idk,
            "idw":    idw,
            "data_r": data_r,
            "cena":   cena,
            "stat":   stat,
            "metoda": metoda,
        })

print(f"rezerwacje dodane")

rezerwacje dodane


### Opinie_Klientow

In [216]:
with engine.begin() as conn:
    pairs = conn.execute(text("""
        SELECT DISTINCT id_klienta, id_wyprawy
        FROM   Rezerwacje
    """)).all()

with engine.begin() as conn:
    next_id = conn.execute(text("""
        SELECT COALESCE(MAX(id_oceny), 0) + 1
        FROM   Opinie_Klientow
    """)).scalar()

def losuj_ocene():
    raw = random.normalvariate(4.5, 1)             
    x   = max(0, min(5, raw))                      
    return Decimal(str(round(x * 2) / 2))         

kom_pos = [
    "Fantastyczne przeżycie!",
    "Polecam każdemu",
    "Widoki nie z tej Ziemi",
    "Obsługa na medal",
    "Najlepsza przygoda w życiu"
]

kom_neg = [
    "Spodziewałem/am się czegoś więcej…",
    "Zbyt drogo jak na jakość usługi",
    "Problemy organizacyjne popsuły wrażenia",
    "Nie czułem/am się bezpiecznie",
    "Raczej nie polecam tej wyprawy"
]

oceny = []   # (id_oceny, id_klienta, id_wyprawy, ocena, komentarz, czy_poleca)

for id_klienta, id_wyprawy in pairs:
    rating = losuj_ocene()
    poleca = int(rating >= Decimal("3.5"))        

    komentarz = random.choice(kom_pos if poleca else kom_neg)

    oceny.append((
        next_id, id_klienta, id_wyprawy,
        rating, komentarz, poleca
    ))
    next_id += 1

with engine.begin() as conn:
    conn.execute(text("""
        INSERT INTO Opinie_Klientow (
            id_oceny, id_klienta, id_wyprawy,
            ocena, komentarz, czy_poleca
        ) VALUES (
            :ido, :idk, :idw,
            :oc, :kom, :poleca
        )
    """), [
        {
            "ido":    ido,
            "idk":    idk,
            "idw":    idw,
            "oc":     oc,
            "kom":    kom,
            "poleca": polec
        }
        for (ido, idk, idw, oc, kom, polec) in oceny
    ])

print(f"oceny klientów dodane")


oceny klientów dodane


In [217]:

# with engine.begin() as conn:
#     conn.execute(text("DELETE FROM Oceny_Klientow"))
    
# print("Tabela Oceny_Klientow została wyczyszczona")

### Wspieramy

In [218]:

def enum_values(table, column):
    with engine.begin() as conn:
        raw = conn.execute(text(f"""
            SELECT COLUMN_TYPE
            FROM   information_schema.COLUMNS
            WHERE  TABLE_SCHEMA = DATABASE()
              AND  TABLE_NAME   = '{table}'
              AND  COLUMN_NAME  = '{column}'
        """)).scalar()
    return [s.strip("'") for s in raw[5:-1].split(",")]

formy_pomocy = enum_values("Wspieramy", "forma_pomocy")

id_f = 1
nazwy = [
    "Fundacja Kosmiczne Jutro", "Zielona Orbita",
    "Stellar Aid", "Galaktyczny Darczyńca", "AstroEco",
]
kraje = ["Polska", "USA", "Niemcy", "Japonia", "Australia"]

fundacje = []


for i in range(len(nazwy)):
    nazwa = nazwy[i]
    kraj = kraje[i]
    forma = random.choice(formy_pomocy)
    data = datetime(2024, 1, 1) + timedelta(days=random.randint(0, 900))  

    fundacje.append((id_f, nazwa, kraj, forma, data))
    id_f += 1


with engine.begin() as conn:
    for (idf, nazwa, kraj, forma, data) in fundacje:
        conn.execute(text("""
            INSERT INTO Wspieramy
                (id_fundacji, nazwa, kraj, forma_pomocy, data_rozpoczecia_wspolpracy)
            VALUES
                (:idf, :nazwa, :kraj, :forma, :data)
        """), {
            "idf": idf,
            "nazwa": nazwa,
            "kraj": kraj,
            "forma": forma,
            "data": data,
        })

print(f"{len(fundacje)} fundacji dodano ")


5 fundacji dodano 


### Partnerzy_Startegiczni

In [219]:

def enum_values(table, column):
    with engine.begin() as conn:
        raw = conn.execute(text(f"""
            SELECT COLUMN_TYPE
            FROM   information_schema.COLUMNS
            WHERE  TABLE_SCHEMA = DATABASE()
              AND  TABLE_NAME   = '{table}'
              AND  COLUMN_NAME  = '{column}'
        """)).scalar()
    return [s.strip("'") for s in raw[5:-1].split(",")]

statusy = enum_values("Partnerzy_Strategiczni", "status_wspolpracy")



nazwy = [
    "SpaceTech Inc.", "Galactic Systems", "Lunar Robotics",
    "AstroData Corp.", "Orion Aerospace", "DeepSpace AI"
]
branze = ["IT", "kosmiczna", "zbrojeniowa", "edukacja", "badawcza", "automatyka"]

partnerzy = []
id_p = 1

for i in range(len(nazwy)):
    nazwa = nazwy[i]
    branza = random.choice(branze)
    email = f"kontakt@{nazwa.lower().replace(' ', '')}.com"
    status = random.choice(statusy)

    partnerzy.append((id_p, nazwa, branza, email, status))
    id_p += 1


with engine.begin() as conn:
    for (idp, nazwa, branza, email, status) in partnerzy:
        try:
            conn.execute(text("""
                INSERT INTO Partnerzy_Strategiczni
                    (id_partnera, nazwa, branża, kontakt_email, status_wspolpracy)
                VALUES
                    (:idp, :nazwa, :branza, :email, :status)
            """), {
                "idp": idp,
                "nazwa": nazwa,
                "branza": branza,
                "email": email,
                "status": status,
            })
        except Exception as e:
            print(f"Błąd przy ID={idp}: {e}")

print(f"{len(partnerzy)} partnerów dodano")


6 partnerów dodano


### Sponsorzy

In [220]:

def enum_values(table, column):
    with engine.begin() as conn:
        raw = conn.execute(text(f"""
            SELECT COLUMN_TYPE
            FROM   information_schema.COLUMNS
            WHERE  TABLE_SCHEMA = DATABASE()
              AND  TABLE_NAME   = '{table}'
              AND  COLUMN_NAME  = '{column}'
        """)).scalar()
    return [s.strip("'") for s in raw[5:-1].split(",")]


typy_sponsorow = enum_values("Sponsorzy", "typ")



nazwy = [
    "Stacja Orzeł", "KosmoPol", "DeepInvest", "Fundusz Galaktyczny",
    "NovaFuture", "CosmoGroup", "SkyPatron"
]
kraje = ["Polska", "USA", "Niemcy", "Francja", "Chiny", "Korea Południowa", "Norwegia"]

sponsorzy = []
id_s = 1

for i in range(len(nazwy)):
    nazwa = nazwy[i]
    kraj = random.choice(kraje)
    typ = random.choice(typy_sponsorow)
    kwota = Decimal(str(random.randint(100_000, 5_000_000))).quantize(Decimal("0.01"))
    data = datetime(2022, 1, 1) + timedelta(days=random.randint(0, 900)) 

    sponsorzy.append((id_s, nazwa, kraj, typ, kwota, data))
    id_s += 1


with engine.begin() as conn:
    for (ids, nazwa, kraj, typ, kwota, data) in sponsorzy:
        try:
            conn.execute(text("""
                INSERT INTO Sponsorzy
                    (id_sponsora, nazwa, kraj, typ, kwota_wsparcia, data_poczatku_wspolpracy)
                VALUES
                    (:ids, :nazwa, :kraj, :typ, :kwota, :data)
            """), {
                "ids": ids,
                "nazwa": nazwa,
                "kraj": kraj,
                "typ": typ,
                "kwota": kwota,
                "data": data,
            })
        except Exception as e:
            print(f"Błąd przy sponsorze {ids}: {e}")

print(f"{len(sponsorzy)} sponsorów dodano")


7 sponsorów dodano
